In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split

# Define data augmentation and normalization for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/home/jovyan/Intelligent_Robotics/Pressed_NonPressed/button/button/'  # Adjust the path to your data directory
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Split dataset into train and val
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply validation transforms to the validation dataset
val_dataset.dataset.transform = data_transforms['val']

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Model and Modify
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)  # Binary classification

model_ft = model_ft.to(device)

criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy with Logits Loss

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

import time
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).float().view(-1, 1)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = torch.sigmoid(outputs) > 0.5

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

# Save the best model weights
torch.save(model_ft.state_dict(), 'best_model_weights.pth')




/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 96.0MB/s]


Epoch 0/24
----------


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


train Loss: 0.8122 Acc: 0.4239
val Loss: 0.6775 Acc: 0.5000

Epoch 1/24
----------
train Loss: 0.5651 Acc: 0.7391
val Loss: 0.4591 Acc: 0.8750

Epoch 2/24
----------
train Loss: 0.4689 Acc: 0.7935
val Loss: 0.3489 Acc: 0.8750

Epoch 3/24
----------
train Loss: 0.4792 Acc: 0.7500
val Loss: 0.3200 Acc: 0.8750

Epoch 4/24
----------
train Loss: 0.3480 Acc: 0.8913
val Loss: 0.3016 Acc: 0.9167

Epoch 5/24
----------
train Loss: 0.3744 Acc: 0.8261
val Loss: 0.2833 Acc: 0.9167

Epoch 6/24
----------
train Loss: 0.3454 Acc: 0.8478
val Loss: 0.2972 Acc: 0.8750

Epoch 7/24
----------
train Loss: 0.2739 Acc: 0.8913
val Loss: 0.3052 Acc: 0.8750

Epoch 8/24
----------
train Loss: 0.2491 Acc: 0.9348
val Loss: 0.2785 Acc: 0.8750

Epoch 9/24
----------
train Loss: 0.2509 Acc: 0.9239
val Loss: 0.2786 Acc: 0.8750

Epoch 10/24
----------
train Loss: 0.2616 Acc: 0.9239
val Loss: 0.2750 Acc: 0.9167

Epoch 11/24
----------
train Loss: 0.2494 Acc: 0.9130
val Loss: 0.2788 Acc: 0.8750

Epoch 12/24
----------
t

In [13]:
from PIL import Image


# Inference
def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB format
    image = preprocess(image).unsqueeze(0)  # Add batch dimension
    return image

def predict(image_path, model):
    image = preprocess_image(image_path)
    image = image.to(device)
    
    with torch.no_grad():
        outputs = model(image)
        prediction = torch.sigmoid(outputs).item()  # Convert logits to probability
        return prediction

# Example usage
image_path = '/home/jovyan/Intelligent_Robotics/Pressed_NonPressed/3.png'  # Replace with the path to your image
prediction = predict(image_path, model_ft)


if prediction > 0.5:
    print('Pressed')
else:
    print('Not Pressed')

0.213651642203331
Not Pressed
